<a href="https://colab.research.google.com/github/GremlinAssassin/AssemblyInfer/blob/main/DirectInjection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This cell sets up the script with the python packages that we both need (numpy and pandas) and thoses that are useful for the way we have designed this script (pathlib) and ones that can make the data look nice (matplotlib and seaborn). We are only going to look at the MSexperiment class from assembly Infer and the ripper converts mzML files into a Json format.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from assemblyinfer import MSExperiment

import mzmlripper.extractor as ripper


ModuleNotFoundError: ignored

Next is the the code that will tell this script where the data you you would like to look at is DATA_FOLDER and where the files should go when we are done. It has been setup so that the finiahed filesand plots will go into the same folder

In [ ]:
DATA_FOLDER = 'D:\Silke\SIA7-59\mzml'

DATA_OUTPUT = DATA_FOLDER

This is the bit that the 'ripper' will do. We are converting mZML files into JSon files. It allows us easier access to the data inside the files. By doing it this way we also check to see if the corresponding JSON file already exsists. If it does then it will not reconvert the file saving a lot of time.

This will check all folders and subfolders in a specific location. If you do NOT want to look in subfolders change rglob to glob.


In [ ]:
for mzml in list(Path(DATA_FOLDER).rglob("*.mzml")):
    
    if not Path(DATA_FOLDER, f'ripper_{mzml.stem}.json').exists():
        
        ripper.process_mzml_file(
            filename=str(mzml),
            out_dir=DATA_FOLDER)

We now have our converted files, we now need to clean up and filter our data. We will 

In [ ]:
for jsn in list(Path(DATA_FOLDER).rglob("*.json")):
    
    print(f"starting analysis of {jsn.stem}")

    msdata = MSExperiment(
        source_file=str(jsn),
        error=0.01
    )
    #filters by intensity on the total ion current. This value will change with instruments.
    msdata.filter_by_tic(tic_threshold=1E6)
    #this then combines MS2 spectra based on the precursor mass 
    msdata.combine_spectra_by_precursor(spectra=msdata.spectra, ms_level=2)


    try:
         peak_count_dict = msdata.get_peak_counts_from_consensus_spectra(int_filter=0.05, occur_filter=0.25)
        
    except Exception:
        print(f"no peaks for {jsn.stem}")
        continue

    #Pandas dataframe with m/x and Peak Count
    peak_count_df = pd.DataFrame(
        list(peak_count_dict.items()), columns=['m/z', 'MS2 Peak Count'])
    
    #data type conversion to numeric (float values)
    peak_count_df = peak_count_df.apply(pd.to_numeric)

    peak_count_df['Sample Name'] = jsn.stem
    all_peak_counts = all_peak_counts.append(peak_count_df, ignore_index=True)

    peak_count_df.to_csv(Path(PLOT_OUTPUT, f"{jsn.stem}_peak_counts.csv"), index=False)



Now we have our data cleaned, filtered and outputted to CSV format we can make a plot to visulise the data.

In [ ]:
    # make basic scatter plot of MS2 peak count and m/z
    plt.scatter(x='m/z', y='MS2 Peak Count', data=peak_count_df)
    
    # add title of json file name
    plt.title(jsn.stem)
    
    # add axis titles
    plt.xlabel("m/z")
    plt.ylabel("MS2 Peak Count")
    
    # this normally makes sure that the full plot is saved and not weirdly chopped
    plt.tight_layout()
            
    # save plot as png
    plt.savefig(Path(PLOT_OUTPUT, f"{jsn.stem}.png"), dpi=300)
    
    # close the plot
    plt.close()